In [ ]:
from typing import Optional, Union, Tuple, List, Callable, Dict
import torch
from diffusers import StableDiffusionPipeline
import torch.nn.functional as nnf
import numpy as np
import abc
import utilsV14
import seq_aligner
import os
import torch.nn as nn
from attention_controller import *
import cv2
import matplotlib.pyplot as plt
import diffusers
from PIL import Image, ImageDraw, ImageFont
from skimage import morphology

print("diffusers:",diffusers.__version__)

#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
MY_TOKEN = ''
LOW_RESOURCE = False
NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 7.5
MAX_NUM_WORDS = 77
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
ldm_stable = StableDiffusionPipeline.from_pretrained("./CompVis/stable-diffusion-v1-4", use_auth_token=MY_TOKEN,torch_dtype = torch.float16).to(device)
# 
#ldm_stable = StableDiffusionPipeline.from_pretrained("./models--stabilityai--stable-diffusion-xl-base-1.0", use_auth_token=MY_TOKEN).to(device)
tokenizer = ldm_stable.tokenizer
output_folder = "./modify_by_reference/experiments"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
#print("ldm_stable: ",ldm_stable)
#ldm_stable = nn.DataParallel(ldm_stable,device_ids=[1],find_unused_parameters=True)

In [ ]:
def run_and_display(prompts, controller, if_process = False, in_img = None, full_img = None,if_save = False, latent=None, run_baseline=False, generator=None,sqr_replace=False,switch = False, bbox = None,img_name = None , output_folder = None):
    if run_baseline:
        images, latent = run_and_display(prompts, EmptyControl(), latent=latent, run_baseline=False, generator=generator)
    images, x_t = utilsV14.text2image_ldm_stable(ldm_stable, prompts, controller, real_img=in_img, full_img = full_img, latent=latent, num_inference_steps=NUM_DIFFUSION_STEPS, guidance_scale=GUIDANCE_SCALE, generator=generator, low_resource=LOW_RESOURCE,is_switch=switch,bboxes=bbox)
    #print("image: ",images.shape)
    #show_cross_attention(controller, res=16, from_where=("up", "down"))
    #utilsV14.view_images(images,if_save, img_name = img_name, output_folder=output_folder)
    return images, x_t

In [ ]:
g_cpu = torch.Generator().manual_seed(8888)

prompts = [""]

controller = AttentionStore()

start_idx = 0
step_len = 4

dir_coco = "./COCO/out_img"
out_dix = "./inversion_exp_out_SDV1-4"
method = "/ours_SD14_half"

out_folder = out_dix+method
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

total_img_num = len(os.listdir(dir_coco))
print("total amount of image is",total_img_num)

name_idx = start_idx
while name_idx < total_img_num:

    obj_name = 'hat'

    in_path = './img_output/' + obj_name + '.jpg'
    out_path = './special_cases' + method + '_' + obj_name + '.jpg'
    
    real_img = cv2.imread(in_path)
    real_img = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)

    real_img_full = np.array(Image.fromarray(real_img).resize((512, 512)))
    real_img_full = np.expand_dims(real_img_full,0)

    image, x_t = run_and_display(prompts, controller, if_save = False, latent=None, run_baseline=False, generator=g_cpu, full_img=real_img_full)
    #show_cross_attention(controller, res=16, from_where=("up", "down"))

    image = np.squeeze(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGRA) 
    cv2.imwrite(out_path,image)
    torch.cuda.empty_cache()
    name_idx = name_idx + step_len
    break